In [1]:
import pandas as pd
import numpy as np
import plotly.plotly as py
from plotly.graph_objs import *
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

In [2]:
df = pd.read_csv('data/enigma-us.states.ca.counties.los-angeles.hcidla.forclosures.2014.csv')
df = df.dropna()

In [3]:
df['property_dist'] = pd.Series()
geolocator = Nominatim()
location = geolocator.geocode("200 N Spring St, Los Angeles")
lach = (location.latitude,location.longitude)
for i in df.index:
    loc = (df.loc[i,'property_lat'], df.loc[i,'property_long'])
    df.loc[i,'property_dist'] = int(vincenty(loc, lach).miles)

In [4]:
df['registered_month'] = pd.Series()
for i in df.index:
    df.loc[i,'registered_month'] = df['registered_date'][i].split('-')[0]+'-'+df['registered_date'][i].split('-')[1]

In [11]:
df_sum = pd.DataFrame(columns=['month','dist','count'])

In [7]:
months = df['registered_month'].unique()
months.sort()
months

array(['2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08'], dtype=object)

In [12]:
counter = 0
for m in months:
    dists = df[df['registered_month']==m]['property_dist'].unique()
    dists.sort()
    for d in dists:
        df_sum.loc[counter,'month'] = m
        df_sum.loc[counter,'dist'] = d
        df_sum.loc[counter,'count'] = len(df[(df['registered_month']==m)&(df['property_dist']==d)])
        counter += 1

In [28]:
data = [
    Scatter3d(
        x=df_sum['month'].tolist(),
        y=df_sum['dist'].tolist(),
        z=df_sum['count'].tolist()
    )
]

layout = Layout(
    title='Numbers of LA foreclosure housing by time and distance',
    scene=Scene(
        xaxis=XAxis(title='Reg. month'),
        yaxis=YAxis(title='Miles from LA City Hall'),
        zaxis=ZAxis(title='Numbers of units')
    )
)

fig = Figure(data=data, layout=layout)

py.iplot(fig, filename='foreclosure_plot')